In [1]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
ox.config(log_console=True, use_cache=True)
from geopy.geocoders import Nominatim
import pickle

ModuleNotFoundError: No module named 'osmnx'

In [2]:
#load stored graph pickle file (here you need to change the file location to Google)
with open('data/graph_berlin_full.obj', 'rb') as fp:
    graph_berlin = pickle.load(fp)

In [3]:
# This gives you all the parks in Berlin
query = "Berlin"
tags = {"leisure":"park"}
parks_bln = ox.geometries.geometries_from_place(query, tags)

/home/philipp_heitmann/WalkieLookie/WalkiLookie/lib/python3.6/site-packages/osmnx/utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/home/philipp_heitmann/WalkieLookie/WalkiLookie/lib/python3.6/site-packages/osmnx/geometries.py:816: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_inner_linestring in list(merged_inner_linestrings):
/home/philipp_heitmann/WalkieLookie/WalkiLookie/lib/python3.6/site-packages/osmnx/geometries.py:816: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry

In [4]:
# Columns to keep 
cols = ["element_type", "osmid", "name", "geometry", "nodes", "ways"]
parks_bln_cln = parks_bln.reset_index().loc[:,cols] #reset index and only keep defined columns

In [5]:
# only keep parks which are a "way" or a "relation" -> for other attractions this may not be necessary
parks_bln_cln = parks_bln_cln.loc[parks_bln_cln.element_type!="node",:]

In [6]:
# Store intermediate results
with open('data/parks.obj', 'wb') as fp:
    pickle.dump(parks_bln_cln, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
# Define function to flatten list of lists (multi level)
def flatten_list(list_of_lists):
    '''Flatten list of lists (multi level) to one list'''
    res = []
    for v in list_of_lists:
        if isinstance(v, list):
            res += flatten_list(v)
        else:
            if isinstance(v, int):
                res.append(v)
    return res  

In [8]:
def retrieve_nodes_within_park(park_data):
  '''Takes one row of the park dataframe and returns dictonary with name and nodes within park'''

  park = park_data.geometry
  name = park_data["name"]
  osmid = park_data.osmid

  park_nds = ox.geometries.geometries_from_polygon(park, {"highway":True, "type":"way"})
  
  try:
    park_nds = park_nds.reset_index()[["element_type", "osmid","geometry","nodes"]] #reset index and only keep relevant columns

    park_nodes_compl = list(flatten_list(park_nds[park_nds.element_type=="node"].osmid)) # add all single nodes to list
    park_nodes_compl += list(flatten_list(park_nds[park_nds.element_type=="way"].nodes)) # add all way nodes to list
    
    return {
      "osmid":osmid,
      "name":name,
      "nodes_within_poi": park_nodes_compl
      }
  except KeyError: #in case of an error / no ways etc. found return same columns, but set column nodes within park to np.nan
    return {
      "osmid":osmid,
      "name":name,
      "nodes_within_poi": np.nan
      } 

!!! **Attention**: The code below is the one which takes for ever as we call the OSM API individually for each park -> one API cal per minute ca. 
- Therefore we should check the shape of the dataframe beforehand !!!!
- I think we can improve the code, so that instead of calling "apply" we do a for loop where we iterate over the whole dataframe and store each result per row in Drive (pickle or csv), so that we in theory could stop the algorithm in between and restart later without having to start all over again
- In the end we then only combine all the intermediate results 

In [ ]:
#parks_final = parks_bln_cln.apply(retrieve_nodes_within_park, axis=1, result_type="expand")

In [10]:
# Store result as csv
parks_final.to_csv("data/parks_bln_final.csv")

In [11]:
# Store result as csv
with open('data/parks_final.obj', 'wb') as fp:
    pickle.dump(parks_bln_cln, fp, protocol=pickle.HIGHEST_PROTOCOL)

Now compare the nodes within the park against the street graph

In [24]:
bln_parks_ = parks_final.copy()

In [25]:
bln_parks_.columns

Index(['osmid', 'name', 'nodes_within_poi'], dtype='object')

In [23]:
# list of nodes on street graph
gr_bln = set(graph_berlin.nodes())

In [61]:
# calculate number of nodes
bln_parks_["no_nodes"] = bln_parks_.nodes_within_poi.apply(lambda x: len(str(x)))

In [49]:
# Keep parks with more than 3 nodes (I think I did that because if the API does not find a node it still gives "NaN" as a string back therefore I filtered it )
bln_parks_cln = bln_parks_[bln_parks_.no_nodes>3].copy()

In [50]:
# Create new column which only contains nodes per park which are on the street graph
bln_parks_cln["nodes_on_graph"]= bln_parks_cln.nodes_within_poi.apply(lambda x:[node for node in x if node in gr_bln])

In [51]:
bln_parks_cln.sort_values("no_nodes", ascending=True)

,osmid,name,nodes_within_poi,no_nodes,nodes_on_graph
1878,752691414.0,NaN,"[30816701, 30816702]",20,"[30816701, 30816702]"
1169,362222483.0,NaN,"[29711819, 249742306]",21,"[29711819, 249742306]"
1146,338392302.0,NaN,"[29985684, 311509739]",21,"[29985684, 311509739]"
1941,796490177.0,NaN,"[390512745, 538429247]",22,"[390512745, 538429247]"
1934,787957934.0,NaN,"[667415983, 667415981]",22,"[667415983, 667415981]"
...,...,...,...,...,...
2249,3099987.0,Volkspark Friedrichshain,"[448814808, 448814809, 8025876349, 8025876350,...",23723,"[3243911112, 385384089, 102374416, 7370245102,..."
1455,549688688.0,Gärten der Welt,"[9592974502, 9592974618, 9592974780, 959297499...",27641,"[282279941, 8989749002, 1500538187, 1500538097..."
493,51807947.0,Britzer Garten,"[2413500340, 2413500353, 2413500356, 241350037...",29326,"[650617214, 650617217, 650617222, 6203325072, ..."
2298,7317281.0,Tempelhofer Feld,"[727341442, 731644262, 5454115321, 5492275322,...",34544,"[727341442, 731644262, 731830450, 5463053931, ..."


In [56]:
bln_parks_cln["number_nodes_on_graph"] = bln_parks_cln.nodes_on_graph.apply(lambda x: len(x))

In [57]:
bln_parks_cln.sort_values("nodes_on_graph", ascending=True)

,osmid,name,nodes_within_poi,no_nodes,nodes_on_graph,number_nodes_on_graph
658,99146926.0,Amalienpark,"[9247171235, 9247171236, 9247171225, 584712544...",562,[],0
1028,255788499.0,NaN,"[92824564, 9721400576, 622296758, 9810987032, ...",568,[],0
1046,266087429.0,NaN,"[2716871818, 2716871851, 2716871853, 271687185...",264,[],0
590,74817865.0,NaN,"[883509849, 883510547, 883515584, 883515585, 8...",353,[],0
1044,265788229.0,NaN,"[2988094442, 2988094445]",24,[],0
...,...,...,...,...,...,...
894,175616015.0,Humannplatz,"[9854280899, 665453535, 660397684, 661285233, ...",1064,"[9854280899, 665453535, 660397684, 661285233, ...",93
134,24750758.0,Waldeckpark,"[8138790080, 9779674666, 9779674667, 977967466...",747,"[9855052414, 1437624993, 268981755, 268981736,...",47
567,69245894.0,Erich-Weinert-Park,"[9855742966, 9855742971, 9855742967, 985574297...",282,"[9855742966, 9855742971, 9855742967, 985574297...",24
158,25362058.0,NaN,"[9855830896, 5590521655, 5590521656, 985583088...",394,"[9855830896, 5590521655, 5590521656, 985583088...",33


In [59]:
# Filter out empty parks
bln_parks_cln[bln_parks_cln.number_nodes_on_graph!=0]

,osmid,name,nodes_within_poi,no_nodes,nodes_on_graph,number_nodes_on_graph
3,4413796.0,Preußenpark,"[8248462102, 8248462103, 8248462104, 824846210...",3146,"[260939007, 260939013, 279150993, 260939019, 2...",259
4,4440110.0,NaN,"[559488592, 559488601, 5320259898, 5320259900,...",130,"[559488592, 559488601, 5320259898, 5320259900,...",11
5,4582243.0,Volkspark Wilmersdorf,"[9506059079, 9506060033, 9506070307, 950607820...",2042,"[7012262008, 1326831874, 254986438, 105388325,...",166
6,4582244.0,Volkspark Wilmersdorf,"[270676050, 3357945131, 74862755, 559229043, 5...",1275,"[270676050, 3357945131, 74862755, 559229043, 5...",103
7,4592806.0,Fritz Schloß-Park,"[331993856, 3301693219, 26992220, 92901871, 13...",5822,"[331993856, 3301693219, 26992220, 92901871, 13...",497
...,...,...,...,...,...,...
2350,14121667.0,NaN,"[340411373, 340411484, 340411264, 340411256, 2...",112,"[340411373, 340411484, 340411264, 340411256, 2...",10
2351,14235419.0,East-Side-Park,"[476571126, 476571133, 476571121, 476571134, 4...",867,"[476571126, 476571133, 476571121, 476571134, 4...",74
2352,14236112.0,Schlesischer Busch,"[5865991167, 29787346, 1934347106, 29787347, 2...",2691,"[29787346, 1934347106, 29787347, 262005337, 16...",231
2353,14282253.0,Wassertorplatz Nord,"[3875882034, 3058137110, 2925266614, 292526661...",400,"[3875882034, 3058137110, 2925266614, 292526661...",34


In [67]:
bln_parks_cln.loc[bln_parks_cln.number_nodes_on_graph!=0, ["osmid", "name", "nodes_on_graph", "number_nodes_on_graph"]].to_csv("data/parks_bln_final_clean.csv")

In [68]:
tst = pd.read_csv("data/parks_bln_final_clean.csv")

In [69]:
tst

,Unnamed: 0,osmid,name,nodes_on_graph,number_nodes_on_graph
0,3,4413796.0,Preußenpark,"[260939007, 260939013, 279150993, 260939019, 2...",259
1,4,4440110.0,NaN,"[559488592, 559488601, 5320259898, 5320259900,...",11
2,5,4582243.0,Volkspark Wilmersdorf,"[7012262008, 1326831874, 254986438, 105388325,...",166
3,6,4582244.0,Volkspark Wilmersdorf,"[270676050, 3357945131, 74862755, 559229043, 5...",103
4,7,4592806.0,Fritz Schloß-Park,"[331993856, 3301693219, 26992220, 92901871, 13...",497
...,...,...,...,...,...
2002,2350,14121667.0,NaN,"[340411373, 340411484, 340411264, 340411256, 2...",10
2003,2351,14235419.0,East-Side-Park,"[476571126, 476571133, 476571121, 476571134, 4...",74
2004,2352,14236112.0,Schlesischer Busch,"[29787346, 1934347106, 29787347, 262005337, 16...",231
2005,2353,14282253.0,Wassertorplatz Nord,"[3875882034, 3058137110, 2925266614, 292526661...",34
